In [2]:
# ==============================
# Facial Emotion Recognition (Lightweight)
# FER2013 + MobileNetV2 (TensorFlow/Keras)
# ==============================

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import pandas as pd
import numpy as np